In [ ]:
import sqlite3
from datetime import date
hot_date = date.today()
DATABASE = "chessdelite.db"
db_connection = sqlite3.connect(DATABASE) 
chess_db = db_connection.cursor()

## Database commands for ad hoc use

In [ ]:
chess_db.close()
db_connection.close()

In [ ]:
db_connection.rollback()

In [ ]:
db_connection.commit()

In [ ]:
chess_db.execute("DROP TABLE xx")

## FEN Header table 
### Creation and population
These are the standard types of pgn header fields. The original/required/standard ones are given importances of 0 or 1; the fancier/optional/more recent ones have a priority of 2.

In [ ]:
chess_db.execute('''CREATE TABLE FEN_Headers
             (FEN_Headers CHARACTER PRIMARY KEY, FEN_Header_Importance INTEGER)''')

In [ ]:
header_fields = [['Event', 1], ['Site', 1], ['White', 1], ['Black', 1], ['WhiteElo', 2], 
                 ['BlackElo', 2], ['Result', 1], ['TimeControl', 2], ['Date', 1],
                ['EventDate', 0], ['EventType', 0], ['Round', 1], ['ECO', 2], ['EventCountry', 0], 
                 ['EventRounds', 0], ['PlyCount', 2], ['Source', 0], ['SourceDate', 0],
                ['Mode', 0], ['FEN', 0], ['Time', 0], ['Termination', 0], ['Annotator', 2]]

In [ ]:
for header_tag in header_fields:
    chess_db.execute("""INSERT INTO FEN_Headers 
    VALUES ('%s', '%s')"""  %(header_tag[0], header_tag[1]))

In [ ]:
db_connection.commit()

## Game Stubs table
### This is the table of pgn headers for each game. 
At the moment, the program uses these to create the displayed header for each game. It would be nice for all the StubHeaderField values to match the list of FEN_Headers in the table above, but pgn praxis allows them to be arbitrary, and every provider seems to create their own.

In [ ]:
chess_db.execute('''CREATE TABLE Gamestubs
             (StubGameKey INTEGER, StubHeaderField CHARACTER, StubHeaderValue CHARACTER)''')

In [ ]:
db_connection.commit()

## Games table

In [ ]:
chess_db.execute('''CREATE TABLE Games
             (GameKey INTEGER PRIMARY KEY, GameWhitePlayer INTEGER, GameBlackPlayer INTEGER, 
             GameSource INTEGER, GameResult INTEGER, GameSourceType INTEGER, GameRound INTEGER)''')

In [ ]:
# Artificial Player keys for flagging me and repertoires
me_key = 1
white_rep = 5
black_rep = 6

Slav_parent_node = 1
KID_parent_node = 7
english_parent_node = 27

DUMMY_PLAYER = 0 
DUMMY_GAME_SOURCE = 0
GAME_UNDECIDED = -2
GAME_SOURCE_TYPE_REPERTOIRE = 2
NO_GAME_ROUND = 0

In [ ]:
# Creating a game tuple for a repertoire "game" to cover the English opening
game_tuple = (DUMMY_PLAYER, DUMMY_PLAYER, DUMMY_GAME_SOURCE, GAME_UNDECIDED, GAME_SOURCE_TYPE_REPERTOIRE, NO_GAME_ROUND)
chess_db.execute("""INSERT INTO Games (GameWhitePlayer, GameBlackPlayer, GameSource, GameResult, GameSourceType, GameRound)
    VALUES ('%s', '%s', '%s', '%s', '%s', '%s')"""  %(game_tuple))

In [ ]:
db_connection.commit()

## Positions table
### Create table and multi-column index

In [ ]:
chess_db.execute('''CREATE TABLE Positions
             (PositionKey INTEGER PRIMARY KEY, PositionRank8 CHARACTER, PositionRank7 CHARACTER, 
             PositionRank6 CHARACTER, PositionRank5 CHARACTER, PositionRank4 CHARACTER, PositionRank3 CHARACTER, 
             PositionRank2 CHARACTER, PositionRank1 CHARACTER, PositionWhoseMove CHARACTER, 
             PositionCastlingPrivileges CHARACTER)''')

In [ ]:
chess_db.execute('''CREATE INDEX position_idx 
    ON Positions (PositionRank8, PositionRank7, PositionRank6, PositionRank5, PositionRank4, PositionRank3, 
             PositionRank2, PositionRank1, PositionWhoseMove, PositionCastlingPrivileges);''')

### Create first row for the table, the initial position of the game

In [ ]:
initial_position = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'

In [ ]:
def split_fen(hotFEN: str) -> tuple:
    fen_list = hotFEN.split()
    fen_whose_move = fen_list[1]
    fen_castling_privileges = fen_list[2]
    fen_rows = fen_list[0]
    return (fen_rows, fen_whose_move, fen_castling_privileges)

In [ ]:
def get_tuple_for_position_table(positionFEN: str):
    fen_rows_string, fen_whose_move, fen_castling_privileges = split_fen(positionFEN)
    fen_rows_list = fen_rows_string.split("/")
    fen_rows_list.append(fen_whose_move) 
    fen_rows_list.append(fen_castling_privileges) 
    return tuple(fen_rows_list)

In [ ]:
initial_position_fields = get_tuple_for_position_table(initial_position)  
initial_position_fields

In [ ]:
chess_db.execute("""INSERT INTO Positions (PositionRank8, PositionRank7, PositionRank6, PositionRank5, PositionRank4, 
        PositionRank3, PositionRank2, PositionRank1 , PositionWhoseMove, PositionCastlingPrivileges)
    VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')"""  %(initial_position_fields))

In [ ]:
db_connection.commit()

## Moves table

In [ ]:
chess_db.execute('''CREATE TABLE Moves
             (MoveKey INTEGER PRIMARY KEY, MoveGame INTEGER, MoveNumber INTEGER, MoveWhoseMove INTEGER, 
             SquareFrom CHARACTER, SquareTo CHARACTER, MovePromotionPiece CHARACTER, 
             LineLevel CHARACTER, PositionFrom INTEGER, PositionTo INTEGER,
             MoveParent INTEGER, LineParent INTEGER, SublineIndex INTEGER)''')

In [ ]:
db_connection.commit()

## Opening Nodes table

In [ ]:
chess_db.execute('''CREATE TABLE OpeningNodes
             (NodeKey INTEGER PRIMARY KEY, NodeName CHARACTER, NodeAbbrev CHARACTER,  
             NodePositionKey INTEGER, NodeParentNode INTEGER, NodeStemGame INTEGER, 
             NodeIsRoot INTEGER)''')

In [ ]:
db_connection.commit()